# RandomForest, XGBoost, LGBMClassifier, CatBoost Ensemble

안녕하세요 ISCA 입니다.  
  
저희는 RandomForest, XGBoost, LGBMClassifier, CatBoost 모델 4개를 Ensemble 하여 모델 집합을 생성한 후, 그 모델 집합들을 다른 데이터 셋(4개)으로 각각 학습하여 최종 3개의 모델 집합들을 Ensemble을 진행하였습니다. 마지막 데이터에 대해서는 성능적 측면에서 학습에 제외를 하였습니다.  
즉, Ensemble 기법을 2번 사용한 것이라고 생각하면 될 것 같습니다.(도합 12개의 각각의 모델 사용)  
  
유감이게도, 최종 예측을 확률이 아닌 1 또는 0 인 최종 분류로 제출을 해서 비록 성능이 제대로 측정되지는 않았습니다.(저희 팀 모두 아쉽게 생각하는 부분입니다.)  
다음에는 이런 실수를 하지 않도록 배울 수 있었던 기회가 된 것 같습니다.  
  
여러 모델개발을 시도하면서 머신러닝 개발에 대한 많은 지식을 얻을 수 있었던 기회였다고 생각됩니다

## 데이터 셋 생성을 위한 코드
데이터 셋은 Feature Engeenering 기법을 가미하여 총 4개를 생성하였습니다.  
  
생성된 4개의 데이터 셋 중 3개의 데이터를 위의 모델 집합들을 학습하는데 한개씩 이용하였습니다.  
(4개의 데이테 셋에 대한 자세한 내용은 이후 단계에서 한개씩 언급하도록 하겠습니다.)

In [1]:
# library import 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import svm 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# data read
# Need to set up a flexiable data file path
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

### 결측치 처리
데이터에는 결측치가 존재합니다.  
이런 결측치를 저희는 각 feature의 "중간값"을 이용하여 처리했습니다.  
  
country feature 에 대해서는 drop을 하였습니다.

In [3]:
# missing value processing
# replace with median
train.fillna(train.median(), inplace=True)  
test.fillna(train.median(), inplace=True)

# drop country feature
train.drop(['country'], axis=1, inplace=True)
test.drop(['country'], axis=1, inplace=True)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14400\3929533334.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train.fillna(train.median(), inplace=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14400\3929533334.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  test.fillna(train.median(), inplace=True)


In [4]:
# Checking processed data
display(train[:10])

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
0,0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,...,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,...,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,...,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,...,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,...,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0
5,5,5.0,4.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,...,1.0,26,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
6,6,4.0,3.0,4.0,3.0,5.0,4.0,5.0,4.0,5.0,...,2.0,40,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1
7,7,4.0,5.0,4.0,4.0,4.0,4.0,2.0,5.0,3.0,...,2.0,34,1.0,2.0,5.0,1.0,1.0,2.0,2.0,1
8,8,4.0,4.0,3.0,4.0,4.0,5.0,4.0,3.0,3.0,...,1.0,20,1.0,7.0,1.0,1.0,1.0,3.0,2.0,0
9,9,3.0,3.0,4.0,3.0,4.0,2.0,4.0,2.0,4.0,...,2.0,17,1.0,10.0,1.0,2.0,1.0,5.0,2.0,0


In [5]:
# train_data_split
# train_input : train feature data
# train_target : train target data
train_input = train.iloc[:, :-1]
train_target = train.iloc[:, -1]

In [6]:
# check train_input data
display(train_input[:10])

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,gender,engnat,age,hand,religion,orientation,voted,married,familysize,ASD
0,0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,...,3.0,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0
1,1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,...,2.0,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0
2,2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,...,1.0,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0
3,3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,...,1.0,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0
4,4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,...,2.0,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0
5,5,5.0,4.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,...,1.0,1.0,26,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,6,4.0,3.0,4.0,3.0,5.0,4.0,5.0,4.0,5.0,...,2.0,2.0,40,1.0,1.0,1.0,2.0,1.0,1.0,2.0
7,7,4.0,5.0,4.0,4.0,4.0,4.0,2.0,5.0,3.0,...,2.0,2.0,34,1.0,2.0,5.0,1.0,1.0,2.0,2.0
8,8,4.0,4.0,3.0,4.0,4.0,5.0,4.0,3.0,3.0,...,2.0,1.0,20,1.0,7.0,1.0,1.0,1.0,3.0,2.0
9,9,3.0,3.0,4.0,3.0,4.0,2.0,4.0,2.0,4.0,...,1.0,2.0,17,1.0,10.0,1.0,2.0,1.0,5.0,2.0


In [7]:
# check train_target data
display(train_target[:10])

0    1
1    1
2    1
3    1
4    0
5    1
6    1
7    1
8    0
9    0
Name: nerdiness, dtype: int64

### 데이터 정규화
이후 모든 실행 코드는 함수 단위로 작성되어 있습니다.  
  
아래 함수는 생성된 데이터에 대해서 표준 정규화를 수행합니다.

In [8]:
# standard scaler
def scaler(dataset):
  scaler = StandardScaler()
  out = scaler.fit_transform(dataset)
  return out

### 데이터를 생성 작업

#### full_data
전체데이터를 모두 사용합니다.(country feature 제거한 table)  
전체데이터에서 범주형 데이터에 해당하는 feature은 one hot encoding 작업을 통해 transform 시켰습니다.

In [9]:
# full data generate function
def full_data(dataset):
  full_data = pd.get_dummies(dataset, columns=['gender', 'hand', 'religion', 'orientation', 'voted', 'married', 'ASD'], drop_first=True)
  full_data = scaler(full_data)
  return full_data

#### 마키아벨리즘 Feature Engineering
해당 데이터 들이 마키아벨리즘 테스트와 연관이 있어 관련 정보를 이용하여 Feature Engineering을 진행하였습니다.  

[참고자료 : https://dacon.io/codeshare/1711]

In [10]:
# feature Engineering - MV data
def MV_data(dataset):
  MV_data = dataset.iloc[:, [1,2,3,4,5,7,8,9,10,11,12,13,15,16,17,19,20]]

  for i in [3,7,10,16,4,11,17,9]:
    MV_data['Q' + str(i)] = 6 - MV_data['Q' + str(i)]

  MV_data['score'] = np.sum(MV_data, axis=1)

  MV_data = scaler(MV_data)

  return MV_data

#### Survey data
모델을 학습했을 때 성능이 비교적 우수하게 나왔던 Q1 ~ Q26에 대한 데이터들만 따로 추출하여 데이터 셋을 만들었습니다.

In [11]:
# survey data - Q1~Q26
def survey_data(dataset):
  survey_data = dataset.iloc[:, 1:26]
  survey_data = scaler(survey_data)
  return survey_data

#### pca data
위에서 생성된 survey 데이터를 sklearn 내장 클래스인 PolynomialFeatures 를 이용하여 추가적인 특징을 생성한 다음 pca 차원축소알고리즘을 이용하여 50개로 축소하였습니다.  
그러나 해당 데이터는 모델의 전반적인 성능과 균형을 고려하여 사용을 하지 않았습니다.

In [12]:
# pca data - maximized dimention about input data and decrese of dimention
# input : survey data
def pca_data(dataset):
  poly = PolynomialFeatures(include_bias=False, degree=2)
  poly_data = poly.fit_transform(dataset)

  pca = PCA(n_components=50)
  pca_data = pca.fit_transform(poly_data)
  return pca_data

## 모델 개발
맨 위 인트로에서 말했던 모델 부분입니다.  
아래 리스트에 적혀있는 4개의 모델들을 1개의 모델 집합으로 생성하였습니다.
- 모델 집합은 각 데이터 셋(4개)당 1개씩 3개 생성하였습니다.
  - XGBoostClassifier
  - LGBMClassifier
  - RandomForestClassifier
  - CatboostClassifier
  
*catboost는 colab에 기본 설치된 모델이 아니므로 추가적인 설치과정이 필요합니다.*

In [13]:
# install catboost
!pip install catboost

#### 모델 집합 생성

역시 모델의 훈련을 한번에 진행할 수 있도록 함수로 작성을 합니다.  
4개의 모델은 예측한 결과(0 또는 1)을 투표 형식으로 Ensemble하여 모델 집합의 최종 결론을 0 또는 1로 내립니다.

In [14]:
# Functions that create and predict modelsfrom sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier, Pool

def model_fit_predict(dataset, targetdata, testset):
  model1 = RandomForestClassifier()
  model2 = XGBClassifier(n_estimators=2000, learning_rate=0.03, max_depth=20)
  model3 = LGBMClassifier(n_estimators=2000, max_depth=20)
  model4 = CatBoostClassifier(iterations=2, depth=7, learning_rate=0.03, 
                              loss_function='Logloss', verbose=True)


  # fit
  model1.fit(dataset,targetdata)
  model2.fit(dataset,targetdata)
  model3.fit(dataset,targetdata)
  model4.fit(dataset,targetdata)
    
  # predict
  pred1 = model1.predict(testset)
  pred2 = model2.predict(testset)
  pred3 = model3.predict(testset)
  pred4 = model4.predict(testset)

  pred_sum = pred1 + pred2 + pred3 + pred4

  # voting ensemble
  predict_final = []
  for pred in pred_sum:
    if pred > 2:
      predict_final.append(1)
    else:
      predict_final.append(0)

  return np.array(predict_final)

### 데이터 생성
이전에 만든 함수들로 실제 데이터들을 설정합니다.

In [15]:
# generating data
full_data_train = full_data(train_input)
full_data_test = full_data(test)

MV_data_train = MV_data(train_input)
MV_data_test = MV_data(test)

survey_data_train = survey_data(train_input)
survey_data_test = survey_data(test)

# not using
pca_data_train = pca_data(survey_data_train)
pca_data_test = pca_data(survey_data_test)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14400\1655268469.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MV_data['Q' + str(i)] = 6 - MV_data['Q' + str(i)]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14400\1655268469.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MV_data['score'] = np.sum(MV_data, axis=1)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14400\1655268469.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

### 실제 모델 생성
모델 집합을 생성해주는 함수와 생성한 데이터들을 이용하여 각각 데이터에 맞추어진 모델을 생성합니다.  
각각의 모델은 model**n**_pred라는 이름으로 생성됩니다.  
  
일단 모델은 5개를 생성했지만 실 사용은 "model1_pred, model2_pred, model3_pred" 입니다.

In [16]:
model1_pred = model_fit_predict(full_data_train, train_target, full_data_test)

0:	learn: 0.6852859	total: 210ms	remaining: 210ms
1:	learn: 0.6777590	total: 258ms	remaining: 0us


In [17]:
model2_pred = model_fit_predict(MV_data_train, train_target, MV_data_test)

0:	learn: 0.6878181	total: 2.18ms	remaining: 2.18ms
1:	learn: 0.6828444	total: 4.24ms	remaining: 0us


In [18]:
model3_pred = model_fit_predict(survey_data_train, train_target, survey_data_test)

0:	learn: 0.6856796	total: 17.8ms	remaining: 17.8ms
1:	learn: 0.6781058	total: 21.8ms	remaining: 0us


### 모델 테스트

생성한 모델에 대해서 예측 성능 테스트를 진행했습니다.  
위에서 생성한 model4와 model5 는 이후 Ensemble에는 사용하지 않았습니다.

In [19]:
# model predict
# Disable Model4 <- comment
print(model1_pred[:10])
print(model2_pred[:10])
print(model3_pred[:10])

[0 1 1 1 1 0 1 1 0 0]
[0 1 1 1 1 0 1 1 0 0]
[0 1 1 1 1 0 1 1 0 0]


### 실제 예측값 만들기
model1, model2, model3 에 해당하는 값들을 일단 모두 더합니다.  
  
그 후 3개의 모델 집합 중 2개 이상의 모델 집합이 판단하는 쪽으로 예측을 진행합니다.

In [20]:
preds = model1_pred + model2_pred + model3_pred

In [21]:
# final model Ensemble
predict_final = []
for pred in preds:
  if pred >= 1.5:
    predict_final.append(1)
  else:
    predict_final.append(0)

result = np.array(predict_final)

## 제출 파일 생성

In [22]:
# generate submission file
submission = pd.read_csv('sample_submission.csv')
submission['nerdiness'] = result
submission.to_csv("submission.csv", index = False)